In [1]:
import os
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'

In [2]:
# 使用huggingface库加载Qwen-7B-Chat模型 https://huggingface.co/Qwen/Qwen-7B-Chat
from transformers import AutoModelForCausalLM, AutoTokenizer
device = "cuda" # the device to load the model onto

# Model names: "Qwen/Qwen-7B-Chat"
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen-7B-Chat", trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen-7B-Chat",
    device_map="auto",
    trust_remote_code=True
).eval()

/home/hujili/.conda/envs/rag/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
The model is automatically converting to bf16 for faster inference. If you want to disable the automatic precision, please manually add bf16/fp16/fp32=True to "AutoModelForCausalLM.from_pretrained".
Try importing flash-attention for faster inference...
Loading checkpoint shards: 100%|██████████| 8/8 [00:05<00:00,  1.44it/s]


In [3]:
from llama_index import VectorStoreIndex, SimpleDirectoryReader
from llama_index import ServiceContext
from llama_index import LLMPredictor

import torch
from typing import Optional, List, Mapping, Any
from transformers import pipeline
from llama_index import ServiceContext, SimpleDirectoryReader, SummaryIndex
from llama_index.callbacks import CallbackManager
from llama_index.llms import (
    CustomLLM,
    CompletionResponse,
    CompletionResponseGen,
    LLMMetadata,
)
from llama_index.llms.base import llm_completion_callback
from transformers import AutoModelForCausalLM, AutoTokenizer

In [4]:
class OurLLM(CustomLLM):
    context_window: int = 2048
    num_output: int = 256
    model_name: str = "Qwen-7B-Chat"
    @property
    def metadata(self) -> LLMMetadata:
        """Get LLM metadata."""
        return LLMMetadata(
            context_window=self.context_window,
            num_output=self.num_output,
            model_name=self.model_name,
        )

    @llm_completion_callback()
    def complete(self, prompt: str, **kwargs: Any) -> CompletionResponse:
        prompt_length = len(prompt)

        # only return newly generated tokens
        text,_ = model.chat(tokenizer, prompt, history=[])
        return CompletionResponse(text=text)

    @llm_completion_callback()
    def stream_complete(
        self, prompt: str, **kwargs: Any
    ) -> CompletionResponseGen:
        raise NotImplementedError()

In [5]:
from llama_index.embeddings import resolve_embed_model
# 指定本地嵌入模型路径
local_embed_model_path = "/home/hujili/code/model/m3e-base"
# 解析本地嵌入模型
embed_model = resolve_embed_model(f"local:{local_embed_model_path}")

In [6]:
llm = OurLLM()
service_context = ServiceContext.from_defaults(llm=llm,embed_model=embed_model)


[nltk_data] Downloading package punkt to /tmp/llama_index...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [8]:
from llama_index import VectorStoreIndex, SimpleDirectoryReader

documents = SimpleDirectoryReader("./data").load_data()
index = VectorStoreIndex.from_documents(documents,service_context=service_context)

In [9]:
query_engine = index.as_query_engine()
response = query_engine.query("Java泛型是什么？")
print(response)

Java泛型是一种功能强大的工具，它可以帮助我们创建更安全、可重用的代码。当我们声明一个类或者方法时，如果我们在参数或返回类型上使用了泛型，则在编译时JVM会强制检查该类型的正确性，避免了类型不匹配导致的运行时异常。另外，泛型还可以使我们的代码更加简洁和易于理解，因为我们在声明类型时并不需要每次都显式地指定具体的数据类型。Java泛型的主要优点包括提高了代码的复用率、减少了类型的错误和提高了代码的可读性。在上面的例子中，`CollectionTest1`类就是一个使用了泛型的例子，它创建了一个`HashSet`并添加了两个`Person`对象，其中`Person`是一个泛型类，可以接收任意类型的参数。


In [10]:
from llama_index.query_engine import RetrieverQueryEngine

#构建查询引擎
base_retriever = index.as_retriever(similarity_top_k=3)
query_engine2 = RetrieverQueryEngine.from_args(base_retriever, service_context=service_context)
query_engine2.query("Java数据类型有哪些？")

Response(response="Java主要有以下几种基本数据类型：\n1. byte：用于表示一个字节的数据，范围为-128到127。\n2. short：用于表示一个短字节的数据，范围为-32768到32767。\n3. int：用于表示一个整数的数据，范围为-2147483648到2147483647。\n4. long：用于表示一个长整数的数据，范围为-9223372036854775808到9223372036854775807。\n5. float：用于表示单精度浮点数的数据，范围为3.4e-45到3.4e+38。\n6. double：用于表示双精度浮点数的数据，范围为1.7e-308到1.7e+308。\n7. char：用于表示一个Unicode字符的数据，范围为'\\u0000'到'\\uffff'。\n8. boolean：用于表示一个逻辑值的数据，只能取值true或false。\n\n除了上述的基本数据类型外，Java还有引用类型，包括类、接口、数组和字符串。它们是通过对象来操作的，并且可以存储任何类型的值。", source_nodes=[NodeWithScore(node=TextNode(id_='981820b0-73ed-40f9-b554-795e3a6e1e29', embedding=None, metadata={'file_path': 'data/java.doc', 'creation_date': '2024-03-02', 'last_modified_date': '2024-03-02', 'last_accessed_date': '2024-03-02'}, excluded_embed_metadata_keys=['creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='b952be9d-dcaa-492d-8d